In [8]:
import copy
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import networkx
from ItalySetup import ItalySetupProvinces
from covidOCP import COVIDVaccinationOCP, COVIDParametersOCP
from main import pick_scenario, build_scenario
import seaborn as sns
import datetime
import pickle
import os

nx = 9
states_names = ['S', 'E', 'P', 'I', 'A', 'Q', 'H', 'R', 'V']
when = 'future-mobintime'
file_prefix = f'week'
outdir = 'helvetios-runs/2021-11-05-107_90/'

generated_dir = 'model_output/2021-11-05'

n_int_steps = 50
nnodes = 107  # nodes
ndays_ocp = 90
ndays = 90

setup = ItalySetupProvinces(nnodes, ndays, when)
setup_ocp = ItalySetupProvinces(nnodes, ndays_ocp, when)
M = setup.nnodes
N = len(setup.model_days) - 1

with open(f'italy-data/parameters_{nnodes}_{when}.pkl', 'rb') as inp:
    p = pickle.load(inp)
    
os.makedirs(f'{generated_dir}', exist_ok=True)

scenarios = {pick_scenario(setup, i)['name']:pick_scenario(setup, i) for i in np.arange(15)}
scenarios.keys()

Loaded Italy Provincial Setup with 107 nodes.
Loaded Italy Provincial Setup with 107 nodes.


dict_keys(['U-r3-t479700-id0', 'L-r3-t479700-id1', 'U-r15-t125000-id2', 'L-r15-t125000-id3', 'U-r15-t250000-id4', 'L-r15-t250000-id5', 'U-r15-t479700-id6', 'L-r15-t479700-id7', 'U-r15-t1000000-id8', 'L-r15-t1000000-id9', 'U-r15-t1500000-id10', 'L-r15-t1500000-id11', 'U-r15-t2000000-id12', 'L-r15-t2000000-id13', 'U-r150-t479700-id14'])

In [9]:
# Choose a subset of scenarios:
pick = 'r15-'
scenarios = {k:v for (k,v) in scenarios.items() if pick in k}
print(len(scenarios))

12


In [10]:
scenarios_opt = {}
scenarios_baseline = {}
## Re-integrate vacc
for scenario_name, scenario in scenarios.items():
    fname = f"{outdir}{file_prefix}-{scenario_name}-opt-{nnodes}_{ndays_ocp}.csv"  # '-'.join(scenario_name.split('-')[:-1])
    try:
        md = pd.read_csv(fname, index_col= 'date', parse_dates=True)
        print(f'YES {fname}')
    
        # Build scenario
        maxvaccrate_regional, delivery_national, stockpile_national_constraint, control_initial = build_scenario(setup, scenario)
        M = setup.nnodes
        N = setup.ndays - 1
        control_initial = np.zeros((M, N))
        unvac_nd = np.copy(setup.pop_node)
        stockpile = 0
        for k in range(ndays_ocp - 1):
            stockpile += delivery_national[k]
            for nodename in md.place.unique():
                nd = setup.ind2name.index(nodename)
                to_allocate = md[(md['place'] == nodename) & (md['comp'] == 'vacc')].iloc[k]['value']
                to_allocate = min(to_allocate, maxvaccrate_regional[nd, k], unvac_nd[nd], stockpile)
                control_initial[nd, k] = to_allocate
                stockpile -= to_allocate
                unvac_nd[nd] -= to_allocate
        p.apply_epicourse(setup, scenario['beta_mult'])
        # END Build scenario

        results, state_initial, yell, mob = COVIDVaccinationOCP.integrate(N,
                                                                          setup=setup,
                                                                          parameters=p,
                                                                          controls=control_initial,
                                                                          save_to=f'{generated_dir}/{scenario_name}-opi-{nnodes}_{ndays}',
                                                                          n_rk4_steps=n_int_steps)
        results.set_index('date', drop=True, inplace=True)
        scenarios_opt[scenario_name] = scenario
        print(f'--> DONE {scenario_name}')

        if scenario_name.split('-')[0] not in scenarios_baseline:
            control_initial = np.zeros((M, N))
            # Generate NO vaccination scenarios
            results, state_initial, yell, mob = COVIDVaccinationOCP.integrate(N,
                                                                      setup=setup,
                                                                      parameters=p,
                                                                      controls=control_initial,
                                                                      save_to=f'{generated_dir}/{scenario_name}-novacc-{nnodes}_{ndays}',
                                                                      n_rk4_steps=n_int_steps)
            scenarios_baseline[scenario_name.split('-')[0]] = pd.read_csv(f'{generated_dir}/{scenario_name}-novacc-{nnodes}_{ndays}.csv', 
                                                                         index_col= 'date', parse_dates=True)


    except FileNotFoundError:
        print(f'NOT {fname}')

YES helvetios-runs/2021-11-05-107_90/week-U-r15-t125000-id2-opt-107_90.csv


  1%|          | 1/89 [00:00<00:22,  3.86it/s]

===> Integrating for model_output/2021-11-05/U-r15-t125000-id2-opi-107_90


  1%|          | 1/89 [00:00<00:22,  3.94it/s]

--> DONE U-r15-t125000-id2
===> Integrating for model_output/2021-11-05/U-r15-t125000-id2-novacc-107_90


...

--> DONE U-r15-t2000000-id12
YES helvetios-runs/2021-11-05-107_90/week-L-r15-t2000000-id13-opt-107_90.csv


  1%|          | 1/89 [00:00<00:21,  4.07it/s]

===> Integrating for model_output/2021-11-05/L-r15-t2000000-id13-opi-107_90


100%|██████████| 89/89 [00:22<00:00,  3.88it/s]


--> DONE L-r15-t2000000-id13


In [25]:
# REload if restart because of python multiprocessing bug : https://github.com/jupyter/jupyter_client/issues/541
import glob
baseline_fn = glob.glob(f'{generated_dir}/*novacc*.csv')

scenarios_baseline = {}
for fn in baseline_fn:
    scenarios_baseline[fn.split('-')[2][-1]] = pd.read_csv(fn, index_col= 'date', parse_dates=True)
    md = pd.read_csv(fn, index_col= 'date', parse_dates=True)
baseline_fn

['model_output/2021-11-05/L-r15-t125000-id3-novacc-107_90.csv',
 'model_output/2021-11-05/U-r15-t125000-id2-novacc-107_90.csv']

In [26]:
def build_scenario_proportional(scenario_name, scenario, value_df, method_name):
    #index of value df = node_name
    maxvaccrate_regional, delivery_national, stockpile_national_constraint, control_initial = build_scenario(setup, scenario)
    M = setup.nnodes
    N = setup.ndays - 1
    control_initial = np.zeros((M, N))
    unvac_nd = np.copy(setup.pop_node)
    stockpile = 0
    for k in range(ndays_ocp-1):
        stockpile += delivery_national[k]
        for nodename in value_df.sort_values('value', ascending=False).index:
            nd = setup.ind2name.index(nodename)
            to_allocate = stockpile * value_df.loc[nodename]['value'] / value_df['value'].sum()
            to_allocate = min(to_allocate, maxvaccrate_regional[nd, k], unvac_nd[nd], stockpile)
            control_initial[nd, k] = to_allocate
            stockpile -= to_allocate
            unvac_nd[nd] -= to_allocate
    p.apply_epicourse(setup, scenario['beta_mult'])
    # END Build scenario

    results, state_initial, yell, mob = COVIDVaccinationOCP.integrate(N,
                                                                      setup=setup,
                                                                      parameters=p,
                                                                      controls=control_initial,
                                                                      save_to=f'{generated_dir}/{scenario_name}-{method_name}-{nnodes}_{ndays}',
                                                                      n_rk4_steps=n_int_steps)
    results.set_index('date', drop=True, inplace=True)


def build_scenario_focused(scenario_name, scenario, value_df, method_name):
    # index of value_df is node_name
    # Build scenario
    maxvaccrate_regional, delivery_national, stockpile_national_constraint, control_initial = build_scenario(setup, scenario)
    M = setup.nnodes
    N = setup.ndays - 1
    control_initial = np.zeros((M, N))
    unvac_nd = np.copy(setup.pop_node)
    stockpile = 0
    for k in range(ndays_ocp-1):
        stockpile += delivery_national[k]
        for nodename in value_df.index:
            nd = setup.ind2name.index(nodename)
            to_allocate = maxvaccrate_regional[nd, k]
            to_allocate = min(to_allocate, maxvaccrate_regional[nd, k], unvac_nd[nd], stockpile)
            control_initial[nd, k] = to_allocate
            stockpile -= to_allocate
            unvac_nd[nd] -= to_allocate
    p.apply_epicourse(setup, scenario['beta_mult'])
    # END Build scenario    
    results, state_initial, yell, mob = COVIDVaccinationOCP.integrate(N,
                                                                      setup=setup,
                                                                      parameters=p,
                                                                      controls=control_initial,
                                                                      save_to=f'{generated_dir}/{scenario_name}-{method_name}-{nnodes}_{ndays}',
                                                                      n_rk4_steps=n_int_steps)
    results.set_index('date', drop=True, inplace=True)


def generate_all_alternatives(scenario_name, scenario):
    
    nv = scenarios_baseline[scenario_name.split('-')[0]]
    
    pop_node_df = pd.DataFrame(setup.pop_node, index=setup.ind2name, columns=['value'])
    
    incid = nv[nv['comp'].isin(['yell'])].groupby('place').sum()
    incidpop = incid/pop_node_df
    incid.sort_values('value', ascending=False, inplace=True)
    incidpop.sort_values('value', ascending=False, inplace=True)
    
    suscep = nv[nv['comp'].isin(['S'])].loc[str(setup.start_date)]
    suscep.set_index('place', inplace=True)
    susceppop = suscep/pop_node_df
    suscep.sort_values('value', ascending=False, inplace=True)
    susceppop.sort_values('value', ascending=False, inplace=True)
    
    pop_node_df.sort_values('value', ascending=False, inplace=True)

    # BY INCIDENCE
    build_scenario_focused(scenario_name=scenario_name, scenario=scenario, value_df=incid, method_name='finc');
    build_scenario_proportional(scenario_name=scenario_name, scenario=scenario, value_df=incid, method_name='pinc');
    
    # BY susceptibilty
    build_scenario_focused(scenario_name=scenario_name, scenario=scenario, value_df=suscep, method_name='fsus');
    build_scenario_proportional(scenario_name=scenario_name, scenario=scenario, value_df=suscep, method_name='psus');
    
    # same but perpop
    build_scenario_focused(scenario_name=scenario_name, scenario=scenario, value_df=incidpop, method_name='fincpp');
    build_scenario_proportional(scenario_name=scenario_name, scenario=scenario, value_df=incidpop, method_name='pincpp');
    build_scenario_focused(scenario_name=scenario_name, scenario=scenario, value_df=susceppop, method_name='fsuspp');
    build_scenario_proportional(scenario_name=scenario_name, scenario=scenario, value_df=susceppop, method_name='psuspp');
    
    # BY POPULATION 
    build_scenario_proportional(scenario_name=scenario_name, scenario=scenario, value_df=pop_node_df, method_name='ppop');
    build_scenario_focused(scenario_name=scenario_name, scenario=scenario, value_df=pop_node_df, method_name='fpop');

    #TODO: Centrality based and R0 based
    return True

In [27]:
#generate_all_alternatives(scenario_name, scenario)

In [28]:
# Generate alternatives:
import multiprocessing as mp
pool = mp.Pool(mp.cpu_count())
    
all_sims = pool.starmap(generate_all_alternatives, [(scenario_name, scenario) for scenario_name, scenario in scenarios.items()])

===> Integrating for model_output/2021-11-05/U-r15-t125000-id2-finc-107_90


  0%|          | 0/89 [00:00<?, ?it/s]

===> Integrating for model_output/2021-11-05/L-r15-t1000000-id9-finc-107_90
===> Integrating for model_output/2021-11-05/L-r15-t479700-id7-finc-107_90


  0%|          | 0/89 [00:00<?, ?it/s]

===> Integrating for model_output/2021-11-05/L-r15-t1500000-id11-finc-107_90===> Integrating for model_output/2021-11-05/U-r15-t1000000-id8-finc-107_90



...

  2%|▏         | 2/89 [00:01<00:53,  1.63it/s]]

===> Integrating for model_output/2021-11-05/U-r15-t1500000-id10-fpop-107_90


  0%|          | 0/89 [00:00<?, ?it/s]

===> Integrating for model_output/2021-11-05/U-r15-t1000000-id8-fpop-107_90


100%|██████████| 89/89 [00:59<00:00,  1.49it/s]
Process ForkPoolWorker-43:
Process ForkPoolWorker-41:
Process ForkPoolWorker-38:
Process ForkPoolWorker-39:
Process ForkPoolWorker-36:
Process ForkPoolWorker-40:
Process ForkPoolWorker-44:
Process ForkPoolWorker-34:
Process ForkPoolWorker-33:
Process ForkPoolWorker-35:
Process ForkPoolWorker-37:
Process ForkPoolWorker-48:
Process ForkPoolWorker-45:
Process ForkPoolWorker-42:
Process ForkPoolWorker-47:
Process ForkPoolWorker-46:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):


# Generate the averted cases for all posterior draws

In [29]:
import matlab.engine

eng = matlab.engine.start_matlab()
#eng.cd('matlab/check-strats/', nargout=0)

#eng.workspace['timesV'] = matlab.double(np.arange(738167, 738167+ndays).tolist()) # THIS IS FOR JANUARY 11th

eng.cd('matlab/post-reviews-update/check-strats-ages/', nargout=0)

eng.workspace['timesV'] = matlab.double(np.arange(738160, 738160+ndays).tolist()) # THIS IS FOR JANUARY 4th

In [30]:
scn_results = pd.DataFrame(columns=['newdoseperweek', 'method', 'infected'])
methods = ['opi', 'finc', 'pinc', 'fsus', 'psus','fincpp', 'pincpp', 'fsuspp', 'psuspp', 'ppop', 'fpop'] # 'opt'

for met in methods:
    for scenario_name, scenario in scenarios.items():
        filename = f'{generated_dir}/{scenario_name}-{met}-{nnodes}_{ndays}.csv'
        if os.path.isfile(filename):
            print(f'Doing {filename}')
            maxvaccrate_regional, stockpile_national, stockpile_national_constraint, control_initial = build_scenario(setup, scenario)
            p.apply_epicourse(setup, scenario['beta_mult'])
            
            md = pd.read_csv(filename, index_col= 'date', parse_dates=True)
            
            dosesV = md[md['comp'] == 'vacc'].pivot(columns='placeID', values='value').to_numpy()

            eng.workspace['Vdoses'] = matlab.double(dosesV.tolist())
            eng.workspace['beta_ratio'] = matlab.double(scenario['beta_mult'][0].tolist())
            eng.run('main_script.m', nargout=0)
            ens_exposed = np.array(eng.eval('ens_exposed_preprocess')).flatten()
        
            scn_results = pd.concat([scn_results, pd.DataFrame.from_dict({'newdoseperweek':[int(scenario_name.split('-')[2][1:])]*len(ens_exposed),
                                                                           'method': [met]*len(ens_exposed),
                                                                           'infected':ens_exposed.tolist(),
                                                                           'post_sample':np.arange(len(ens_exposed)),
                                                                           'doses': [dosesV.sum()]*len(ens_exposed),
                                                                           'scenario-beta': [scenario_name.split('-')[0]]*len(ens_exposed),
                                                                           'scenario-rate': [scenario_name.split('-')[1]]*len(ens_exposed),
                                                                           'scenario-tot': [scenario_name.split('-')[2]]*len(ens_exposed),
                                                                           'scenario': [scenario_name]*len(ens_exposed)
                                                                         })])
        else:
            print(f'not found {scenario_name}')

Doing model_output/2021-11-05/U-r15-t125000-id2-opi-107_90.csv
Doing model_output/2021-11-05/L-r15-t125000-id3-opi-107_90.csv
Doing model_output/2021-11-05/U-r15-t250000-id4-opi-107_90.csv
Doing model_output/2021-11-05/L-r15-t250000-id5-opi-107_90.csv
Doing model_output/2021-11-05/U-r15-t479700-id6-opi-107_90.csv
Doing model_output/2021-11-05/L-r15-t479700-id7-opi-107_90.csv
Doing model_output/2021-11-05/U-r15-t1000000-id8-opi-107_90.csv
Doing model_output/2021-11-05/L-r15-t1000000-id9-opi-107_90.csv
Doing model_output/2021-11-05/U-r15-t1500000-id10-opi-107_90.csv
Doing model_output/2021-11-05/L-r15-t1500000-id11-opi-107_90.csv
Doing model_output/2021-11-05/U-r15-t2000000-id12-opi-107_90.csv
Doing model_output/2021-11-05/L-r15-t2000000-id13-opi-107_90.csv
Doing model_output/2021-11-05/U-r15-t125000-id2-finc-107_90.csv
Doing model_output/2021-11-05/L-r15-t125000-id3-finc-107_90.csv
Doing model_output/2021-11-05/U-r15-t250000-id4-finc-107_90.csv
Doing model_output/2021-11-05/L-r15-t25000

MatlabExecutionError: 
  File /Users/chadi/Documents/phd/COVID-19_italy-vaccination-ocp/matlab/post-reviews-update/check-strats-ages/main_script.m, line 34, in main_script

  File /Applications/MATLAB_R2019b.app/toolbox/matlab/lang/run.m, line 91, in run
The end operator must be used within an array index expression.


In [12]:
scenarios_baseline = {}
for scenario_name, scenario in scenarios.items():
    if scenario_name.split('-')[0] not in scenarios_baseline:
        print(scenario_name)

        maxvaccrate_regional, stockpile_national, stockpile_national_constraint, control_initial = build_scenario(setup, scenario)
        
        p.apply_epicourse(setup, scenario['beta_mult'])

        dosesV = np.zeros_like(dosesV)

        eng.workspace['Vdoses'] = matlab.double(dosesV.tolist())
        eng.workspace['beta_ratio'] = matlab.double(scenario['beta_mult'][0].tolist())
        eng.run('main_script.m', nargout=0)
        ens_exposed = np.array(eng.eval('ens_exposed_preprocess')).flatten()
        
        scenarios_baseline[scenario_name.split('-')[0]] = ens_exposed
        

U-r3-t479700-id0
L-r3-t479700-id1


In [13]:
scn_results

,newdoseperweek,method,infected,post_sample,doses,scenario-beta,scenario-rate,scenario-tot,scenario
0,479700,opi,2.698752e+08,0.0,6.318990e+06,U,r3,t479700,U-r3-t479700-id0
1,479700,opi,2.622298e+08,1.0,6.318990e+06,U,r3,t479700,U-r3-t479700-id0
2,479700,opi,2.501296e+08,2.0,6.318990e+06,U,r3,t479700,U-r3-t479700-id0
3,479700,opi,2.545047e+08,3.0,6.318990e+06,U,r3,t479700,U-r3-t479700-id0
4,479700,opi,2.422173e+08,4.0,6.318990e+06,U,r3,t479700,U-r3-t479700-id0
...,...,...,...,...,...,...,...,...,...
97,479700,fpop,8.559696e+07,97.0,6.236100e+06,L,r150,t479700,L-r150-t479700-id15
98,479700,fpop,8.998865e+07,98.0,6.236100e+06,L,r150,t479700,L-r150-t479700-id15
99,479700,fpop,9.195135e+07,99.0,6.236100e+06,L,r150,t479700,L-r150-t479700-id15
100,479700,fpop,8.636084e+07,100.0,6.236100e+06,L,r150,t479700,L-r150-t479700-id15


In [14]:
nbtileU = int(len(scn_results.loc[scn_results['scenario-beta'] == 'U', 'scenario-tot'])/len(scenarios_baseline['U']))
nbtileL = int(len(scn_results.loc[scn_results['scenario-beta'] == 'L', 'scenario-tot'])/len(scenarios_baseline['L']))

In [15]:
scn_results['bl'] = 0

scn_results.loc[scn_results['scenario-beta'] == 'U', 'bl'] = np.tile(scenarios_baseline['U'],nbtileU)
scn_results.loc[scn_results['scenario-beta'] == 'L', 'bl'] = np.tile(scenarios_baseline['L'],nbtileL)

In [16]:
scn_results['averted'] = scn_results['bl'] - scn_results['infected']
scn_results['avertedpervacc'] = scn_results['averted']/scn_results['doses']

In [17]:
scn_results.to_csv(f'{generated_dir}/all_summary.csv')

In [26]:
scenarios_specs = {
'vaccpermonthM': [1.5, 15, 150],  # ax.set_ylim(0.05, 0.4)
# 'vacctotalM': [2, 5, 10, 15, 20],
'newdoseperweek': [125000, 250000, 479700, 1e6, 1.5e6, 2e6],
'epicourse': ['U', 'L']  # 'U'
}

import itertools
# Compute all permutatios
keys, values = zip(*scenarios_specs.items())
permuted_specs = [dict(zip(keys, v)) for v in itertools.product(*values)]

In [27]:
specs = pd.DataFrame.from_dict(permuted_specs)
specs = specs[((specs['vaccpermonthM'] == 15.0) | (specs['newdoseperweek'] == 479700.0))].reset_index(drop = True)
specs

,vaccpermonthM,newdoseperweek,epicourse
0,1.5,479700.0,U
1,1.5,479700.0,L
2,15.0,125000.0,U
3,15.0,125000.0,L
4,15.0,250000.0,U
5,15.0,250000.0,L
6,15.0,479700.0,U
7,15.0,479700.0,L
8,15.0,1000000.0,U
9,15.0,1000000.0,L


In [19]:
specs.to_csv('scenarios_specs_2021-02-12.csv')